In [1]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from flask import Flask
import socketio
import eventlet
import os
import base64
from io import BytesIO
from PIL import Image 

In [ ]:
sio = socketio.Server()

app = Flask(__name__)

speed_limit = 32

def img_preprocess(img):
    
    img = img[50:135,:,:]  # crop to remove noises - final shape (85, 320, 3)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img


@sio.on('telemetry')
def telemetry(sid, data):
    # data: image from center camera
    
    speed = float(data['speed'])
    
    image = Image.open(BytesIO(base64.b64decode(data['image'])))  # decode base64 encoded source image
    image = np.asarray(image)
    image = img_preprocess(image)
    image = np.array([image])
    steering_angle = float(model.predict(image))
    throttle = 1.0 - speed/speed_limit  # 
    print(f"{steering_angle} {throttle} {speed}")
    send_control(steering_angle, 1.0)
    

@sio.on('connect')
def connect(sid, environ):
    print('Connected')
    send_control(0, 0) # initial angle & throttle values

def send_control(steering_angle, throttle):
    sio.emit('steer', data = {
        'steering_angle': steering_angle.__str__(),
        'throttle': throttle.__str__() 
    })

if __name__ == '__main__':
    model = load_model('fin_model.h5')
    app = socketio.Middleware(sio, app)
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(15052) wsgi starting up on http://0.0.0.0:4567
(15052) accepted ('127.0.0.1', 2465)


Connected
-0.04207613691687584 0.972625 0.876
-0.03699594736099243 0.8905 3.504
-0.039945829659700394 0.863125 4.38
-0.08115990459918976 0.994003125 0.1919
-0.08207804709672928 0.97174375 0.9042
-0.07934413105249405 0.9499125 1.6028
-0.07358954101800919 0.93536875 2.0682
-0.06606896221637726 0.9147875 2.7268
-0.07032333314418793 0.900284375 3.1909
-0.07628773897886276 0.8782156249999999 3.8971
-0.07211347669363022 0.85635 4.5968
-0.049319539219141006 0.84183125 5.0614
-0.049319539219141006 0.82736875 5.5242
-0.06291186064481735 0.805771875 6.2153
-0.06691024452447891 0.79144375 6.6738
-0.06293809413909912 0.770059375 7.3581
-0.06293809413909912 0.7488125 8.038
-0.06087207794189453 0.73471875 8.489
-0.06644712388515472 0.7067 9.3856
-0.07724445313215256 0.6927749999999999 9.8312
-0.07621833682060242 0.66509375 10.717
-0.08114136010408401 0.651340625 11.1571
-0.06261737644672394 0.634971875 11.6809
-0.05793946608901024 0.626709375 11.9453
-0.08507825434207916 0.6013593749999999 12.7565
-

0.0570790059864521 0.056603124999999976 30.1887
0.0570790059864521 0.05660937499999996 30.1885
0.04234428331255913 0.05660937499999996 30.1885
0.04234428331255913 0.056634375000000015 30.1877
0.042289506644010544 0.05662812500000003 30.1879
0.042289506644010544 0.056612499999999955 30.1884
0.03606731817126274 0.05660624999999997 30.1886
0.03606731817126274 0.05661562499999995 30.1883
0.02113514393568039 0.056621875000000044 30.1881
0.02113514393568039 0.05661562499999995 30.1883
0.05159415304660797 0.05660937499999996 30.1885
0.05159415304660797 0.05655937499999997 30.1901
0.04862074553966522 0.05661875000000005 30.1882
0.04862074553966522 0.056621875000000044 30.1881
0.0590786412358284 0.05664374999999999 30.1874
0.0590786412358284 0.056603124999999976 30.1887
0.03253522142767906 0.05660624999999997 30.1886
0.058785099536180496 0.056612499999999955 30.1884
0.025652192533016205 0.056612499999999955 30.1884
0.03927503526210785 0.056603124999999976 30.1887
0.04302247241139412 0.056612499

-0.19337010383605957 0.0566875 30.186
-0.1401766985654831 0.05664999999999998 30.1872
-0.1401766985654831 0.056681250000000016 30.1862
-0.10987600684165955 0.056681250000000016 30.1862
-0.1518765389919281 0.05660624999999997 30.1886
-0.13864676654338837 0.05660937499999996 30.1885
-0.13877280056476593 0.056590625000000006 30.1891
-0.14546988904476166 0.056599999999999984 30.1888
-0.14551059901714325 0.056634375000000015 30.1877
-0.09663783758878708 0.056634375000000015 30.1877
-0.12838761508464813 0.056634375000000015 30.1877
-0.12837369740009308 0.05664374999999999 30.1874
-0.023160763084888458 0.05664374999999999 30.1874
-0.023237794637680054 0.05659687499999999 30.1889
0.06278856843709946 0.05661562499999995 30.1883
0.09264276176691055 0.056603124999999976 30.1887
0.08503810316324234 0.05660937499999996 30.1885
0.08504960685968399 0.056659374999999956 30.1869
0.06653185933828354 0.05664374999999999 30.1874
0.06637812405824661 0.056656249999999964 30.187
0.0817437618970871 0.05664374

0.048990506678819656 0.056612499999999955 30.1884
0.04902515187859535 0.056621875000000044 30.1881
0.001378826331347227 0.056640625 30.1875
0.0012520723976194859 0.056590625000000006 30.1891
-0.0384785421192646 0.05659375 30.189
-0.05079588666558266 0.05659375 30.189
-0.050712112337350845 0.056612499999999955 30.1884
-0.09585411846637726 0.05660937499999996 30.1885
-0.11891955882310867 0.056625000000000036 30.188
-0.10290162265300751 0.056612499999999955 30.1884
-0.10266394168138504 0.05665312499999997 30.1871
-0.13098499178886414 0.05666562500000005 30.1867
-0.1380336433649063 0.05663125000000002 30.1878
-0.21411564946174622 0.056621875000000044 30.1881
-0.17296646535396576 0.056659374999999956 30.1869
-0.14931225776672363 0.05664999999999998 30.1872
-0.13813383877277374 0.05677812500000001 30.1831
-0.23453593254089355 0.056746874999999974 30.1841
-0.23447808623313904 0.05660624999999997 30.1886
-0.20014505088329315 0.056603124999999976 30.1887
-0.16816188395023346 0.05679062499999998

...

(15052) accepted ('127.0.0.1', 2473)


Connected
0.22229357063770294 1.0 0.0
0.22229357063770294 1.0 0.0
0.22229357063770294 1.0 0.0
0.18012920022010803 0.9589375 1.314
0.1606302112340927 0.9607875 1.2548
0.14792118966579437 0.986303125 0.4383
0.14792118966579437 0.98835 0.3728
0.17240481078624725 0.983534375 0.5269
0.17660142481327057 0.98595 0.4496
0.1995575726032257 0.98451875 0.4954
0.20313462615013123 0.982190625 0.5699
0.20811642706394196 0.97854375 0.6866
0.20989078283309937 0.9746875 0.81
0.21252071857452393 0.96784375 1.029
0.2156779170036316 0.964121875 1.1481
0.23125213384628296 0.95844375 1.3298
0.2451557219028473 0.9518625000000001 1.5404
0.2491747885942459 0.941296875 1.8785
0.2805926203727722 0.931015625 2.2075
0.291087806224823 0.922775 2.4712
0.2886539399623871 0.9153875 2.7076
0.28312793374061584 0.902121875 3.1321
0.27814775705337524 0.893140625 3.4195
0.259206086397171 0.88343125 3.7302
0.22285553812980652 0.8676375000000001 4.2356
0.20975686609745026 0.85066875 4.7786
0.2078547179698944 0.83243125 5.362

(15052) accepted ('127.0.0.1', 2477)


Connected
0.21896760165691376 0.9863125 0.438
0.21896760165691376 0.9863125 0.438
0.21896760165691376 0.9863125 0.438
0.165553480386734 0.93725 2.008
0.15942226350307465 0.96089375 1.2514
0.14366145431995392 0.98644375 0.4338
0.1575210839509964 0.98846875 0.369
0.17025865614414215 0.983434375 0.5301
0.1872633844614029 0.984371875 0.5001
0.19601809978485107 0.9835125 0.5276
0.20203575491905212 0.97950625 0.6558
0.2069440484046936 0.9764875 0.7524
0.20484299957752228 0.9718875 0.8996
0.2153049111366272 0.967471875 1.0409
0.2210342139005661 0.96128125 1.239
0.23265770077705383 0.9570125 1.3756
0.2429930567741394 0.952903125 1.5071
0.25418007373809814 0.9437656249999999 1.7995
0.279640793800354 0.934453125 2.0975
0.28511932492256165 0.9247125 2.4092
0.28465497493743896 0.9171374999999999 2.6516
0.2833494246006012 0.9090125 2.9116
0.27324944734573364 0.897184375 3.2901
0.2558992803096771 0.888390625 3.5715
0.22448104619979858 0.873746875 4.0401
0.20833173394203186 0.858090625 4.5411
0.20482

127.0.0.1 - - [16/Aug/2020 03:43:39] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 8.623389
127.0.0.1 - - [16/Aug/2020 03:43:39] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 95.054807
127.0.0.1 - - [16/Aug/2020 03:43:39] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 18.422192
